In [1]:
import pandas as pd
import os
import PyPDF2
import re

In [2]:
def extract_text_from_pdf(pdf_path):
    if pdf_path is None or pdf_path.strip() == '':
        return None
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ''
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    except Exception as error:
        print(error)
        return None


def find_all_match_indices(text, pattern):
    regex = re.compile(pattern)
    matches = regex.finditer(text)
    match_indices = [(match.start(), match.end()) for match in matches]
    return match_indices if match_indices else None


In [3]:
results = []
for i, path in enumerate(os.listdir('./crs_reports/')):
    print(f'Path {i}')
    string = extract_text_from_pdf('./crs_reports/' + path)
    if string is None:
        continue
    house_pattern = r'H\.R\.\s{0,3}[0-9]+'
    hr_matches = find_all_match_indices(string, house_pattern)
    if hr_matches is None:
        continue

    for hr_match in hr_matches:
        start, end = hr_match
        mid = (end + start) / 2

        congress_pattern = r'[1]?[901][0-9]th'
        congress_matches = find_all_match_indices(string, congress_pattern)
            
        closest_match = ((None, None), float('inf'))
        if congress_matches:
            for congress_match in congress_matches:
                cstart, cend = congress_match
                cmid = (cend + cstart) / 2

                if abs(cmid - mid) < closest_match[1]:
                    closest_match = ((cstart, cend), abs(cmid - mid))

            closest_start, closest_end = closest_match[0]
            if closest_start is not None:
                congress = string[closest_start : closest_end]
            else:
                congress = None
        else:
            congress = None

        results.append({
            'path': path,
            'match': string[start : end],
            'bill_type': 'hr',
            'congress_match': congress,
            'congress_match_distance': closest_match[1]
        })
            

Path 0
Path 1
Path 2
Path 3
Path 4
Path 5
Path 6
Path 7
Path 8
Path 9
Path 10
Path 11
Path 12
Path 13
Path 14
Path 15
Path 16
Path 17
Path 18
Path 19
Path 20
Path 21
Path 22
Path 23
Path 24
Path 25
Path 26
Path 27
Path 28
Path 29
Path 30
Path 31
Path 32
Path 33
Path 34
Path 35
Path 36
Path 37
Path 38
Path 39
Path 40
Path 41
Path 42
Path 43
Path 44
Path 45
Path 46
Path 47
Path 48
Path 49
Path 50
Path 51
Path 52
Path 53
Path 54
Path 55
Path 56
Path 57
Path 58
Path 59
Path 60
Path 61
Path 62
Path 63
Path 64
Path 65
Path 66
Path 67
Path 68
Path 69
Path 70
Path 71
Path 72
Path 73
Path 74
Path 75
Path 76
Path 77
Path 78
Path 79
Path 80
Path 81
Path 82
Path 83
Path 84
Path 85
Path 86
Path 87
Path 88
Path 89
Path 90
Path 91
Path 92
Path 93
Path 94
Path 95
Path 96
Path 97
Path 98
Path 99
Path 100
Path 101
Path 102
Path 103
Path 104
Path 105
Path 106
Path 107
Path 108
Path 109
Path 110
Path 111
Path 112
Path 113
Path 114
Path 115
Path 116
Path 117
Path 118
Path 119
Path 120
Path 121
Path 122
Pat

In [11]:
df = pd.DataFrame(results)
df.to_csv('crs_matches_with_congresses_12-7-23.csv', index=False)